In [8]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/01_input_history.csv')

In [58]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

def prediction(df, country, product):
    pred = ""
    data = df["Quantity"]
    data = data.to_numpy()
    data = np.reshape(data, [20, 12]) # reshape to divide it into months
    last_row_zero = 0
    for i, d in enumerate(data):
        if (d == 0).all():
            last_row_zero = i + 1
    data = data[last_row_zero:]
    
    if data.shape == (0, 12): # case where we are no longer in production 
        pred = ""
        for i, month in enumerate(["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]):
            pred = pred + f"{country},{product},{month}2024,0\n"
        return pred
    
    data = data.flatten()
    model = SARIMAX(data, order = (1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit()
    pred = ""
    model_predictions = model_fit.forecast(steps = 12)
    for i, month in enumerate(["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]):
        pred = pred + f"{country},{product},{month}2024,{int(model_predictions[i])}\n"
    return pred

In [59]:
c = "Japan"
p = "BrightBreeze Pet Odor Eliminator"
actual_df = df.loc[((df["Country"] == c) & (df["Product"] == p)), ["Month", "Quantity"]]
print(prediction(actual_df, c, p))

Japan,BrightBreeze Pet Odor Eliminator,Jan2024,1980
Japan,BrightBreeze Pet Odor Eliminator,Feb2024,1921
Japan,BrightBreeze Pet Odor Eliminator,Mar2024,1906
Japan,BrightBreeze Pet Odor Eliminator,Apr2024,1895
Japan,BrightBreeze Pet Odor Eliminator,May2024,1891
Japan,BrightBreeze Pet Odor Eliminator,Jun2024,1940
Japan,BrightBreeze Pet Odor Eliminator,Jul2024,1988
Japan,BrightBreeze Pet Odor Eliminator,Aug2024,1971
Japan,BrightBreeze Pet Odor Eliminator,Sep2024,2027
Japan,BrightBreeze Pet Odor Eliminator,Oct2024,1951
Japan,BrightBreeze Pet Odor Eliminator,Nov2024,1973
Japan,BrightBreeze Pet Odor Eliminator,Dec2024,1897



In [60]:
output = open("poly_all_time_model_out.csv", "w+")
output.write("Country,Product,Month,Quantity\n")
countries = df["Country"].unique()
products = df["Product"].unique()
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

for c in countries:
    for p in products:
            actual_df = df.loc[((df["Country"] == c) & (df["Product"] == p)), ["Month", "Quantity"]]
            output.write(prediction(actual_df, c, p))

output.close() # SCORE 948

c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\39346\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All param